# 👊 YOLO11n Fighting/Violence Detection Training

**Multiple Datasets: Roboflow + Kaggle + GitHub**

**Run cells in order!**

## 1. Setup

In [ ]:
# Install packages
!pip install -q ultralytics>=8.3.0 roboflow gdown

# Check GPU
!nvidia-smi

import torch
print(f"\n✅ CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Imports
import os
import glob
import shutil
import yaml
import torch
import zipfile
from ultralytics import YOLO
from IPython.display import display, Image as IPImage

MERGED_PATH = "/content/violence_merged_dataset"
print("✅ Ready!")

## 2. Download Datasets

We'll download from multiple sources:
1. **Roboflow** - Your forked dataset
2. **Kaggle** - Fight detection YOLO dataset
3. **HuggingFace/GitHub** - Public violence datasets

In [ ]:
from roboflow import Roboflow

# ========================================
# 🔑 PASTE YOUR ROBOFLOW API KEY HERE
# ========================================
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
print("✅ Connected to Roboflow!")

In [ ]:
# ========================================
# 📊 DATASET 1: Your forked Roboflow dataset (WORKS!)
# ========================================
try:
    project1 = rf.workspace("securityviolence").project("violence-detection-p4qev")
    dataset1 = project1.version(1).download("yolov8")
    print(f"✅ Dataset 1 (Roboflow): {dataset1.location}")
except Exception as e:
    print(f"⚠️ Dataset 1 failed: {e}")
    dataset1 = None

In [ ]:
# ========================================
# 📊 DATASET 2: Kaggle - Fight Detection YOLO Dataset
# Direct download from public source
# ========================================
import gdown

# Fight Detection Dataset (from HuggingFace - Musawer14/fight_detection_yolov8)
# This is a public dataset with YOLO annotations

print("📥 Downloading Dataset 2 from HuggingFace...")

try:
    !wget -q https://huggingface.co/datasets/Musawer14/fight_detection_yolov8/resolve/main/data.zip -O /content/dataset2.zip
    
    # Extract
    with zipfile.ZipFile('/content/dataset2.zip', 'r') as z:
        z.extractall('/content/dataset2')
    
    dataset2_path = '/content/dataset2'
    print(f"✅ Dataset 2 (HuggingFace): {dataset2_path}")
except Exception as e:
    print(f"⚠️ Dataset 2 failed: {e}")
    dataset2_path = None

In [ ]:
# ========================================
# 📊 DATASET 3: Real Life Violence Situations (Kaggle)
# Extracted frames with violence/non-violence
# ========================================
print("📥 Downloading Dataset 3 (Real Life Violence)...")

try:
    # This is a direct download link to violence detection images
    !wget -q --no-check-certificate 'https://github.com/vdquang1991/TNUE_FightDetection/archive/refs/heads/main.zip' -O /content/dataset3.zip
    
    with zipfile.ZipFile('/content/dataset3.zip', 'r') as z:
        z.extractall('/content/dataset3')
    
    dataset3_path = '/content/dataset3/TNUE_FightDetection-main'
    print(f"✅ Dataset 3 (GitHub): {dataset3_path}")
except Exception as e:
    print(f"⚠️ Dataset 3 failed: {e}")
    dataset3_path = None

In [ ]:
# Check what we have
print("\n" + "="*60)
print("📊 DOWNLOAD SUMMARY")
print("="*60)

if dataset1:
    c1 = len(glob.glob(f"{dataset1.location}/train/images/*")) + len(glob.glob(f"{dataset1.location}/valid/images/*"))
    print(f"Dataset 1 (Roboflow): ✅ {c1} images")
else:
    print("Dataset 1 (Roboflow): ❌")

if dataset2_path and os.path.exists(dataset2_path):
    print(f"Dataset 2 (HuggingFace): ✅ Downloaded")
    !ls {dataset2_path}
else:
    print("Dataset 2 (HuggingFace): ❌")

if dataset3_path and os.path.exists(dataset3_path):
    print(f"Dataset 3 (GitHub): ✅ Downloaded")
    !ls {dataset3_path}
else:
    print("Dataset 3 (GitHub): ❌")

## 3. Prepare & Merge Datasets

In [ ]:
# Create merged directory
for split in ['train', 'valid', 'test']:
    os.makedirs(f"{MERGED_PATH}/images/{split}", exist_ok=True)
    os.makedirs(f"{MERGED_PATH}/labels/{split}", exist_ok=True)

# Class mapping
CLASS_MAP = {
    'violence': 0, 'Violence': 0, 'VIOLENCE': 0,
    'fight': 0, 'Fight': 0, 'fighting': 0, 'Fighting': 0,
    'Violence/Fight': 0, 'violent': 0,
    
    'non_violence': 1, 'NonViolence': 1, 'nonviolence': 1,
    'no_fight': 1, 'NoFight': 1, 'normal': 1, 'Normal': 1,
    'NoViolence/NoFight': 1, 'non-violence': 1
}

CLASSES = {0: 'violence', 1: 'non_violence'}
print(f"✅ Unified Classes: {CLASSES}")

In [ ]:
def merge_roboflow_dataset(ds, prefix):
    """Merge a Roboflow dataset"""
    if ds is None:
        return 0
    
    src = ds.location
    count = 0
    
    yaml_path = f"{src}/data.yaml"
    if not os.path.exists(yaml_path):
        return 0
    
    with open(yaml_path, 'r') as f:
        cfg = yaml.safe_load(f)
    
    src_classes = cfg.get('names', {})
    if isinstance(src_classes, list):
        src_classes = {i: n for i, n in enumerate(src_classes)}
    
    print(f"   Classes: {src_classes}")
    
    for split in ['train', 'valid', 'test']:
        img_dir = f"{src}/{split}/images"
        lbl_dir = f"{src}/{split}/labels"
        
        if not os.path.exists(img_dir):
            continue
        
        for img in glob.glob(f"{img_dir}/*"):
            name = os.path.basename(img)
            base, ext = os.path.splitext(name)
            
            new_name = f"{prefix}_{base}{ext}"
            shutil.copy(img, f"{MERGED_PATH}/images/{split}/{new_name}")
            
            lbl = f"{lbl_dir}/{base}.txt"
            if os.path.exists(lbl):
                with open(lbl, 'r') as f:
                    lines = f.readlines()
                
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        old_id = int(parts[0])
                        old_name = src_classes.get(old_id, '')
                        new_id = CLASS_MAP.get(old_name)
                        if new_id is not None:
                            new_lines.append(f"{new_id} {' '.join(parts[1:])}\n")
                
                if new_lines:
                    with open(f"{MERGED_PATH}/labels/{split}/{prefix}_{base}.txt", 'w') as f:
                        f.writelines(new_lines)
            
            count += 1
    
    return count

In [ ]:
def merge_yolo_folder(src_path, prefix):
    """Merge a YOLO-format dataset from a folder"""
    if src_path is None or not os.path.exists(src_path):
        return 0
    
    count = 0
    
    # Try to find data.yaml
    yaml_files = glob.glob(f"{src_path}/**/data.yaml", recursive=True)
    if not yaml_files:
        yaml_files = glob.glob(f"{src_path}/**/*.yaml", recursive=True)
    
    if yaml_files:
        yaml_path = yaml_files[0]
        base_dir = os.path.dirname(yaml_path)
        
        with open(yaml_path, 'r') as f:
            cfg = yaml.safe_load(f)
        
        src_classes = cfg.get('names', {})
        if isinstance(src_classes, list):
            src_classes = {i: n for i, n in enumerate(src_classes)}
        
        print(f"   Found yaml: {yaml_path}")
        print(f"   Classes: {src_classes}")
        
        for split in ['train', 'valid', 'test', 'val']:
            target_split = 'valid' if split == 'val' else split
            
            # Try different folder structures
            possible_paths = [
                f"{base_dir}/{split}/images",
                f"{base_dir}/images/{split}",
                f"{src_path}/{split}/images",
                f"{src_path}/images/{split}"
            ]
            
            img_dir = None
            for p in possible_paths:
                if os.path.exists(p):
                    img_dir = p
                    break
            
            if not img_dir:
                continue
            
            lbl_dir = img_dir.replace('images', 'labels')
            
            for img in glob.glob(f"{img_dir}/*"):
                name = os.path.basename(img)
                base, ext = os.path.splitext(name)
                
                if ext.lower() not in ['.jpg', '.jpeg', '.png', '.bmp', '.webp']:
                    continue
                
                new_name = f"{prefix}_{base}{ext}"
                shutil.copy(img, f"{MERGED_PATH}/images/{target_split}/{new_name}")
                
                lbl = f"{lbl_dir}/{base}.txt"
                if os.path.exists(lbl):
                    with open(lbl, 'r') as f:
                        lines = f.readlines()
                    
                    new_lines = []
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            old_id = int(parts[0])
                            old_name = src_classes.get(old_id, '')
                            new_id = CLASS_MAP.get(old_name)
                            if new_id is not None:
                                new_lines.append(f"{new_id} {' '.join(parts[1:])}\n")
                    
                    if new_lines:
                        with open(f"{MERGED_PATH}/labels/{target_split}/{prefix}_{base}.txt", 'w') as f:
                            f.writelines(new_lines)
                
                count += 1
    
    return count

In [ ]:
# Merge all datasets
print("🔄 Merging datasets...")
print("="*60)

total = 0

# Dataset 1 - Roboflow
print("\n📦 Processing Dataset 1 (Roboflow)...")
n1 = merge_roboflow_dataset(dataset1, "rf")
print(f"   ✅ Merged {n1} images")
total += n1

# Dataset 2 - HuggingFace
if dataset2_path:
    print("\n📦 Processing Dataset 2 (HuggingFace)...")
    n2 = merge_yolo_folder(dataset2_path, "hf")
    print(f"   ✅ Merged {n2} images")
    total += n2

# Dataset 3 - GitHub
if dataset3_path:
    print("\n📦 Processing Dataset 3 (GitHub)...")
    n3 = merge_yolo_folder(dataset3_path, "gh")
    print(f"   ✅ Merged {n3} images")
    total += n3

print("\n" + "="*60)
print(f"✅ TOTAL MERGED: {total} images")

In [ ]:
# Create data.yaml
data_yaml_path = f"{MERGED_PATH}/data.yaml"

with open(data_yaml_path, 'w') as f:
    yaml.dump({
        'path': MERGED_PATH,
        'train': 'images/train',
        'val': 'images/valid',
        'test': 'images/test',
        'names': CLASSES
    }, f)

# Final stats
train = len(glob.glob(f"{MERGED_PATH}/images/train/*"))
valid = len(glob.glob(f"{MERGED_PATH}/images/valid/*"))

print(f"✅ Created: {data_yaml_path}")
print(f"\n📊 Final Dataset:")
print(f"   Train: {train}")
print(f"   Valid: {valid}")
print(f"   Total: {train + valid}")

## 4. Train Model (YOLO11n)

In [ ]:
# Configuration
MODEL = 'yolo11n.pt'  # nano - fastest for Jetson
EPOCHS = 100
BATCH = 16
IMG_SIZE = 640

PROJECT = 'violence_detection'
NAME = 'yolo11n_violence'

print(f"✅ Model: {MODEL}, Epochs: {EPOCHS}, Batch: {BATCH}")

In [ ]:
# Load model
model = YOLO(MODEL)
print(f"✅ Loaded {MODEL}")

In [ ]:
# Train
print("🚀 Starting training...")

results = model.train(
    data=data_yaml_path,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMG_SIZE,
    patience=30,
    project=PROJECT,
    name=NAME,
    exist_ok=True,
    device=0,
    cache=True,
    amp=True,
    plots=True
)

print("\n✅ Training Complete!")

## 5. Results

In [ ]:
RESULTS_DIR = f"{PROJECT}/{NAME}"

if os.path.exists(f"{RESULTS_DIR}/results.png"):
    display(IPImage(filename=f"{RESULTS_DIR}/results.png"))

In [ ]:
# Validate
best_model = YOLO(f"{RESULTS_DIR}/weights/best.pt")
val = best_model.val(data=data_yaml_path)

print(f"\nmAP50: {val.box.map50:.4f}")
print(f"mAP50-95: {val.box.map:.4f}")

## 6. Export & Download

In [ ]:
# Export ONNX
onnx = best_model.export(format='onnx', imgsz=IMG_SIZE, simplify=True)
print(f"✅ ONNX: {onnx}")

In [ ]:
# Download
from google.colab import files

weights = f"{RESULTS_DIR}/weights"
files.download(f"{weights}/best.pt")

In [ ]:
# Download ONNX
files.download(f"{weights}/best.onnx")

---
## 🎉 Done!

### Deploy on Jetson:
```bash
yolo export model=best.pt format=engine device=0 half=True
```